In [7]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import mlflow
import mlflow.sklearn
import pandas as pd

# Configuración del servidor Flask
app = Flask(__name__)

# Habilitar CORS
CORS(app)

# Configura el URI del servidor MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Cambia según tu servidor MLflow
model_name = "Diabetes"  # Nombre del modelo registrado
model_version = "18"  # Versión del modelo en MLflow

# Cargar el modelo desde el registro de MLflow
try:
    model_uri = f"models:/{model_name}/{model_version}"
    print(f"Cargando el modelo desde {model_uri}...")
    model = mlflow.sklearn.load_model(model_uri)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    model = None  # Manejar casos en los que el modelo no se carga correctamente

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "Tu servidor esta funcionando correctamente y puedes usarlo."})

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Validar que el modelo esté cargado
        if model is None:
            return jsonify({'error': 'Modelo no cargado.'}), 500

        # Obtener datos enviados en formato JSON
        data = request.get_json()

        # Validar que los datos están presentes y tienen el formato correcto
        if not data or not isinstance(data, list):
            return jsonify({'error': 'Datos no válidos. Se esperaba una lista de instancias JSON.'}), 400

        # Convertir los datos en un DataFrame de Pandas
        input_data = pd.DataFrame(data)

        # Validar que el DataFrame no esté vacío
        if input_data.empty:
            return jsonify({'error': 'El conjunto de datos está vacío.'}), 400

        # Realizar la predicción
        predictions = model.predict(input_data)

        return jsonify({'predictions': predictions.tolist()})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Ejecutar la aplicación en el puerto 5001
    app.run(host='127.0.0.1', port=5001, debug=True, use_reloader=False)
    

Cargando el modelo desde models:/Diabetes/18...
Modelo cargado exitosamente.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [20/Jan/2025 10:56:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:56:33] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:56:33] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 11:54:20] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 11:54:21] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 11:54:21] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 11:54:21] "POST /predict HTTP/1.1" 200 -
